In [1]:
import pandas as pd
import numpy as np
from khaiii import KhaiiiApi
from gensim import corpora, models
from pprint import pprint


In [2]:
year = [i for i in range(2017, 2022)]
sortedresult = pd.DataFrame()

In [3]:
for i in range(5):
    print("==== "+str(year[i])+" ====")
    data = pd.read_csv("./modi_data/data_"+str(year[i])+".csv")
    #data = data.drop(['Unnamed: 0'], axis=1)
    
    # dfWordList = pd.read_excel("./khaiii_word_cor.xlsx")
    # dfWordList2 = pd.read_excel("./khaiii_word_cor_etc.xlsx")
    # #print(dfWordList2)

    # dfWordDel = dfWordList[dfWordList["수정"] == "삭제"]
    # dfWordMod = dfWordList[dfWordList["수정"] != "삭제"]
    # dfWordDiv = dfWordList2
    # #print(dfWordMod)

    # seriesDelete = dfWordDel["기존"]
    # stopword = []
    # for word in seriesDelete.values:
    #     stopword.append(word)
    # #print(stopword)

    # seriesModify = dfWordMod["기존"]
    # modiword = []
    # for word in seriesModify.values:
    #     modiword.append(word)
    # #print(len(modiword))

    # seriesModify2 = dfWordMod["수정"]
    # modiword2 = []
    # for word in seriesModify2.values:
    #     modiword2.append(word)
    # #print(len(modiword2))

    # seriesDivide = dfWordDiv["기존"]
    # divword = []
    # for word in seriesDivide.values:
    #     divword.append(word)
    # #print(divword)
    # #print(len(divword))

    # seriesDivide2 = dfWordDiv["수정"]
    # divword2 = []
    # for words in seriesDivide2.values:
    #     divword2.append(words.split(', '))
    # #print(divword2)
    # #print(len(divword2))


    api = KhaiiiApi()
    def khaiiiTokenizer(raw, pos=['NNG', 'NNP', 'NNB', 'NP', 'NR', 'SL']):#, stopword=stopword,): # 일반명사 고유명사 의존명사 대명사 수사 외국어
        list = []
        skip = 0

        for word in api.analyze(raw): #raw data
            #print(word)
            
            for i, morph in enumerate(word.morphs):
                #print(morph.lex)

                if len(morph.lex) > 1 and morph.tag in pos: #and morph.lex not in stopword: 
                    if morph.tag == 'SL':
                        morph.lex = morph.lex.lower()
                    # if morph.lex in divword:
                    #     morph.lex = divword2[divword.index(morph.lex)]
                    #     list.extend(morph.lex)
                    # elif morph.lex in modiword:
                    #     morph.lex = modiword2[modiword.index(morph.lex)]
                    #     list.append(morph.lex)
                    # else: 
                    list.append(morph.lex)
                    
        return list

    tokenized = data['full_data'].apply(lambda row: khaiiiTokenizer(row))
    #print(tokenized)
    #tokenized.to_excel("./finaldata/0911token_full.xls") ##############
    print("========= tokenization completed =========")

    id2word = corpora.Dictionary(tokenized)
    corpus=[id2word.doc2bow(text) for text in tokenized]
    print("# words in total : ", len(id2word))
    print("# documents : ", len(corpus))

    #tfidf
    print("==== calculating tfidf ====")
    tfidf = models.TfidfModel(corpus)

    #tfidf per doc
    tfidflist = []
    for doc in tfidf[corpus]:
        inner_list = [0]*len(id2word) 
        for id, freq in doc:
            inner_list[id] = np.around(freq, decimals=2) #put tfidf value in the place matching its index
        tfidflist.append(inner_list)
    #print(len(tfidflist))

    tfidf_df = pd.DataFrame(tfidflist)
    tfidf_df.columns = [id2word[i] for i in range(len(id2word))] #set columns' names as words
    #print(tfidf_df)

    total_df = pd.concat([data[["date", "id"]], tfidf_df], axis=1)
    #print(total_df)
    total_df.to_csv("./final_data/tfidf"+str(year[i])+".csv") 

    #sum of tfidf for each word
    columnsum = pd.DataFrame(total_df.sum(axis=0)).T
    columnsum = columnsum.drop(['id'], axis=1)
    columnsum['date'] = year[i]
    #print(columnsum)
    columnsum.to_csv("./final_data/sum"+str(year[i])+".csv")

    #sort tfidf value in descending order
    columnsum = columnsum.sort_values(by=0, axis=1, ascending=False)
    print(columnsum)
    columnsum.to_csv("./final_data/sorted"+str(year[i])+".csv")

    print("==== completed ====")

==== 2017 ====
========= tokenization completed =========
# words in total :  27404
# documents :  7275
==== calculating tfidf ====


ValueError: This sheet is too large! Your sheet size is: 7275, 27406 Max sheet size is: 1048576, 16384

In [4]:
# tfidf for full data

data = pd.read_csv("./modi_data/full_data.csv")
#data = data.drop(['Unnamed: 0'], axis=1)
    
# dfWordList = pd.read_excel("./khaiii_word_cor.xlsx")
# dfWordList2 = pd.read_excel("./khaiii_word_cor_etc.xlsx")
# #print(dfWordList2)

# dfWordDel = dfWordList[dfWordList["수정"] == "삭제"]
# dfWordMod = dfWordList[dfWordList["수정"] != "삭제"]
# dfWordDiv = dfWordList2
# #print(dfWordMod)

# seriesDelete = dfWordDel["기존"]
# stopword = []
# for word in seriesDelete.values:
#     stopword.append(word)
# #print(stopword)

# seriesModify = dfWordMod["기존"]
# modiword = []
# for word in seriesModify.values:
#     modiword.append(word)
# #print(len(modiword))

# seriesModify2 = dfWordMod["수정"]
# modiword2 = []
# for word in seriesModify2.values:
#     modiword2.append(word)
# #print(len(modiword2))

# seriesDivide = dfWordDiv["기존"]
# divword = []
# for word in seriesDivide.values:
#     divword.append(word)
# #print(divword)
# #print(len(divword))

# seriesDivide2 = dfWordDiv["수정"]
# divword2 = []
# for words in seriesDivide2.values:
#     divword2.append(words.split(', '))
# #print(divword2)
# #print(len(divword2))


api = KhaiiiApi()
def khaiiiTokenizer(raw, pos=['NNG', 'NNP', 'NNB', 'NP', 'NR', 'SL']):#, stopword=stopword,): # 일반명사 고유명사 의존명사 대명사 수사 외국어
    list = []
    skip = 0

    for word in api.analyze(raw): #raw data
        #print(word)
        
        for i, morph in enumerate(word.morphs):
            #print(morph.lex)

            if len(morph.lex) > 1 and morph.tag in pos: #and morph.lex not in stopword: 
                if morph.tag == 'SL':
                    morph.lex = morph.lex.lower()
                # if morph.lex in divword:
                #     morph.lex = divword2[divword.index(morph.lex)]
                #     list.extend(morph.lex)
                # elif morph.lex in modiword:
                #     morph.lex = modiword2[modiword.index(morph.lex)]
                #     list.append(morph.lex)
                # else: 
                list.append(morph.lex)
                
    return list

tokenized = data['full_data'].apply(lambda row: khaiiiTokenizer(row))
#print(tokenized)
#tokenized.to_excel("./finaldata/0911token_full.xls") ##############
print("========= tokenization completed =========")

id2word = corpora.Dictionary(tokenized)
corpus=[id2word.doc2bow(text) for text in tokenized]
print("# words in total : ", len(id2word))
print("# documents : ", len(corpus))

#tfidf
print("==== calculating tfidf ====")
tfidf = models.TfidfModel(corpus)

#tfidf per doc
tfidflist = []
for doc in tfidf[corpus]:
    inner_list = [0]*len(id2word) 
    for id, freq in doc:
        inner_list[id] = np.around(freq, decimals=2) #put tfidf value in the place matching its index
    tfidflist.append(inner_list)
#print(len(tfidflist))

tfidf_df = pd.DataFrame(tfidflist)
tfidf_df.columns = [id2word[i] for i in range(len(id2word))] #set columns' names as words
#print(tfidf_df)

total_df = pd.concat([data[["date", "id"]], tfidf_df], axis=1)
#print(total_df)
total_df.to_csv("./final_data/tfidf_full.csv") 

#sum of tfidf for each word
columnsum = pd.DataFrame(total_df.sum(axis=0)).T
columnsum = columnsum.drop(['id'], axis=1)
columnsum['date'] = year[i]
#print(columnsum)
columnsum.to_csv("./final_data/sum_full.csv")

#sort tfidf value in descending order
columnsum = columnsum.sort_values(by=0, axis=1, ascending=False)
print(columnsum)
columnsum.to_csv("./final_data/sorted_full.csv")

print("==== completed ====")

========= tokenization completed =========
# words in total :  5086
# documents :  1149
==== calculating tfidf ====
   year     학습  프로그래밍     정보     로봇    컴퓨터    시스템     교육     sw     평가  ...  \
0  2020  38.82  36.43  36.23  33.03  31.51  28.13  26.34  25.67  25.39  ...   

    양육자    소득    가계    영유    오전    오후    장르    주말    주중    영아  
0  0.04  0.04  0.04  0.04  0.04  0.04  0.04  0.04  0.04  0.04  

[1 rows x 5087 columns]
==== completed ====
